In [1]:
import warnings
import pandas as pd
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine import create_engine

#Config
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
class DatabaseSession:
    def __init__(self, db_name: str) -> None:
        self.db_user = 'root'
        self.db_password = 'rushi12345'
        self.db_host = '192.168.29.7'
        self.db_name = db_name
    
    def LocalSession(self) -> sessionmaker:
        engine = create_engine(f"mysql+pymysql://{self.db_user}:{self.db_password}@{self.db_host}/{self.db_name}")
        SessionLocal = sessionmaker(autoflush=True, bind=engine)
        return SessionLocal()

In [3]:
session = DatabaseSession('triggers')
db = session.LocalSession()

In [4]:
class SQL10:
    def __init__(self) -> None:
        pass
    
    def execute(self, query: str) -> None:
        db.execute(text(query))
        print("Success!")
        
    def commit(self) -> None:
        db.execute(text("""commit;"""))
        print("Success!")
        
    def query(self, query: str) -> pd.DataFrame:
        return pd.DataFrame(db.execute(text(query)).fetchall())

In [5]:
data = SQL10()

In [6]:
data.query("""
show tables;
""")

,Tables_in_triggers
0,course
1,course1
2,course_update
3,department
4,employee
5,ref_course


## Triggers

### Triggers in SQL
1) What are Triggers in SQL?
   * Triggers in SQL are special types of stored procedures that are automatically executed (or "triggered") in response to  specific events on a particular table or view in a database. These events can include INSERT, UPDATE, or DELETE operations.

2) Why do we use Triggers in SQL?
   * Triggers are used to enforce business rules, ensure data integrity, and automate certain tasks that need to be performed when specific events occur in the database.

3) Three examples of Triggers in SQL:
   
   * Insert Trigger
   
   ```sql
   CREATE TRIGGER trg_after_insert
   AFTER INSERT ON table_name
   FOR EACH ROW
   BEGIN
    -- Trigger logic
   END;
   ```
   
   * Update Trigger
   
   ```sql
   CREATE TRIGGER trg_after_update
   AFTER UPDATE ON table_name
   FOR EACH ROW
   BEGIN
    -- Trigger logic
   END;
   ```
   
   * Delete Trigger
   
   ```sql
   CREATE TRIGGER trg_after_delete
   AFTER DELETE ON table_name
   FOR EACH ROW
   BEGIN
    -- Trigger logic
   END;
   ```
   
4) Three Potential Use Cases of Triggers in SQL and Data Analytics:
   * Audit Logging: Triggers can be used to log changes made to specific tables, allowing for easy tracking of data modifications over time.
   
   * Data Validation: Triggers can enforce complex business rules or data validation constraints that cannot be enforced using standard constraints.

   * Derived Data: Triggers can be used to automatically update derived data or summary tables when source data changes, improving performance for analytical queries.

In [26]:
data.execute("""
create table course(
course_id int,
course_desc varchar(255),
course_mentor varchar(255),
course_price decimal(10, 2),
course_discount int)
""")

Success!


In [27]:
data.commit()

Sucess!


In [28]:
data.query("""
show tables;
""")

,Tables_in_triggers
0,course
1,course_update


In [30]:
data.execute("""
create table course_update1(
course_mentor_update varchar(255),
course_price_update int,
course_discount_update int);
""")

Success!


In [31]:
data.commit()

Sucess!


In [32]:
data.query("""
show tables;
""")

,Tables_in_triggers
0,course
1,course_update
2,course_update1


* Triggers example

```sql
DELIMITER $$
CREATE TRIGGER course_before_insert
BEFORE INSERT
ON course
FOR EACH ROW
BEGIN
   SET NEW.create_date = SYSDATE();
END $$
DELIMITER ;
```

In [35]:
data.execute("""
ALTER TABLE course ADD COLUMN create_date DATETIME;
""")

Success!


In [37]:
data.execute("""
insert into course (course_id,
course_desc,
course_mentor,
course_price,
course_discount) values
(101, "Full Stack Data Analytics course with hands on projects", "person1", 3999.99, 1000);
""")

Success!


In [38]:
data.query("""select * from course""")

,course_id,course_desc,course_mentor,course_price,course_discount,create_date
0,101,Full Stack Data Analytics course with hands on...,person1,3999.99,1000,2024-03-31 12:18:29


In [39]:
data.execute("""
insert into course (course_id,
course_desc,
course_mentor,
course_price,
course_discount) values
(102, "Full Stack Data Science course with hands on projects", "person2", 6999.99, 1000);
""")

Success!


In [7]:
data.query("""select * from course""")

,course_id,course_desc,course_mentor,course_price,course_discount,create_date
0,101,Full Stack Data Analytics course with hands on...,person1,3999.99,1000,2024-03-31 12:18:29
1,102,Full Stack Data Science course with hands on p...,person2,6999.99,1000,2024-03-31 12:21:45


```sql
DELIMITER $$
CREATE TRIGGER price_based_on_mentor
BEFORE INSERT
ON course
FOR EACH ROW
BEGIN
   SET NEW.course_discount = CASE 
   WHEN NEW.course_mentor = "person3" THEN 0
   ELSE 1000
   END;
END $$
DELIMITER ;
```

In [8]:
data.execute("""
insert into course (course_id,
course_desc,
course_mentor,
course_price) values
(103, "Full Stack Data Science course with hands on projects", "person3", 6999.99);
""")

Success!


In [9]:
data.query("""select * from course""")

,course_id,course_desc,course_mentor,course_price,course_discount,create_date
0,101,Full Stack Data Analytics course with hands on...,person1,3999.99,1000,2024-03-31 12:18:29
1,102,Full Stack Data Science course with hands on p...,person2,6999.99,1000,2024-03-31 12:21:45
2,103,Full Stack Data Science course with hands on p...,person3,6999.99,0,2024-03-31 12:34:34


In [10]:
data.execute("""
insert into course (course_id,
course_desc,
course_mentor,
course_price) values
(104, "Full Stack Data Science course with hands on projects", "person1", 6999.99);
""")

Success!


In [11]:
data.query("""select * from course""")

,course_id,course_desc,course_mentor,course_price,course_discount,create_date
0,101,Full Stack Data Analytics course with hands on...,person1,3999.99,1000,2024-03-31 12:18:29
1,102,Full Stack Data Science course with hands on p...,person2,6999.99,1000,2024-03-31 12:21:45
2,103,Full Stack Data Science course with hands on p...,person3,6999.99,0,2024-03-31 12:34:34
3,104,Full Stack Data Science course with hands on p...,person1,6999.99,1000,2024-03-31 12:34:45


In [12]:
data.execute("""
create table course1(
course_id int,
course_desc varchar(255),
course_mentor varchar(255),
course_price decimal(10, 2),
course_discount int,
course_date datetime,
user_info varchar(255))
""")

Success!


In [13]:
data.commit()

Sucess!


```sql
DELIMITER $$
CREATE TRIGGER course1_before_insert
BEFORE INSERT
ON course1
FOR EACH ROW
BEGIN
   DECLARE user_val varchar(255);
   SET NEW.course_date = SYSDATE();
   SELECT user() into user_val;
   SET NEW.user_info = user_val;
   SET NEW.course_discount = CASE
   WHEN NEW.course_mentor = "person3" THEN 0
   ELSE 1000
   END;
END $$
DELIMITER ;
```

In [14]:
data.execute("""
insert into course1 (course_id,
course_desc,
course_mentor,
course_price) values
(103, "Full Stack Data Science course with hands on projects", "person3", 6999.99);
""")

Success!


In [15]:
data.commit()

Sucess!


In [17]:
data.query("""select * from course1""")

,course_id,course_desc,course_mentor,course_price,course_discount,course_date,user_info
0,103,Full Stack Data Science course with hands on p...,person3,6999.99,0,2024-03-31 12:44:18,root@192.168.29.120


In [18]:
data.execute("""
insert into course1 (course_id,
course_desc,
course_mentor,
course_price) values
(102, "Full Stack Data Science course with hands on projects", "person2", 6999.99);
""")

Success!


In [19]:
data.execute("""
insert into course1 (course_id,
course_desc,
course_mentor,
course_price) values
(101, "Full Stack Data Analytics course with hands on projects", "person1", 3999.99);
""")

Success!


In [20]:
data.commit()

Sucess!


In [21]:
data.query("""select * from course1""")

,course_id,course_desc,course_mentor,course_price,course_discount,course_date,user_info
0,103,Full Stack Data Science course with hands on p...,person3,6999.99,0,2024-03-31 12:44:18,root@192.168.29.120
1,102,Full Stack Data Science course with hands on p...,person2,6999.99,1000,2024-03-31 12:45:17,root@192.168.29.120
2,101,Full Stack Data Analytics course with hands on...,person1,3999.99,1000,2024-03-31 12:45:36,root@192.168.29.120


In [22]:
data.execute("""
create table ref_course(
recored_insert_date datetime,
record_insert_user varchar(255))
""")

Success!


In [23]:
data.commit()

Sucess!


```sql
DELIMITER $$
CREATE TRIGGER course1_ref_before_insert
BEFORE INSERT
ON course1
FOR EACH ROW
BEGIN
   DECLARE user_val varchar(255);
   SET NEW.course_date = SYSDATE();
   SELECT user() into user_val;
   SET NEW.course_discount = 
       CASE
           WHEN NEW.course_mentor = "person3" THEN 0
           ELSE 1000
       END;
   INSERT INTO ref_course values(sysdate(), user_val);
END $$
DELIMITER ;
```

In [9]:
data.execute("""
insert into course1 (course_id,
course_desc,
course_mentor,
course_price) values
(104, "Full Stack Data Analytics course with hands on projects", "person1", 3999.99);
""")

Success!


In [8]:
data.query("""
select * from ref_course;
""")

,recored_insert_date,record_insert_user
0,2024-04-01 10:29:43,root@192.168.29.120


In [11]:
data.commit()

Sucess!


In [9]:
data.execute("""
insert into course1 (course_id,
course_desc,
course_mentor,
course_price) values
(105, "Full Stack Data Science course with hands on projects", "person3", 6999.99);
""")

Success!


In [10]:
data.query("""
select * from ref_course;
""")

,recored_insert_date,record_insert_user
0,2024-04-01 10:29:43,root@192.168.29.120
1,2024-04-01 17:20:44,root@192.168.29.120


In [11]:
data.commit()

Sucess!


In [12]:
data.query("""
select * from course1
""")

,course_id,course_desc,course_mentor,course_price,course_discount,course_date,user_info
0,103,Full Stack Data Science course with hands on p...,person3,6999.99,0,2024-03-31 12:44:18,root@192.168.29.120
1,102,Full Stack Data Science course with hands on p...,person2,6999.99,1000,2024-03-31 12:45:17,root@192.168.29.120
2,101,Full Stack Data Analytics course with hands on...,person1,3999.99,1000,2024-03-31 12:45:36,root@192.168.29.120
3,104,Full Stack Data Analytics course with hands on...,person1,3999.99,1000,2024-04-01 10:29:43,root@192.168.29.120
4,105,Full Stack Data Science course with hands on p...,person3,6999.99,0,2024-04-01 17:20:44,root@192.168.29.120


In [8]:
data.execute("""
create table employee (
e_id int NOT NULL unique,
Name varchar(255),
Salary int,
department_id int)
""")

Success!


In [9]:
data.commit()

Sucess!


In [10]:
data.execute("""
create table department(
id int NOT NULL unique,
Name varchar(255))
""")

Success!


In [11]:
data.commit()

Sucess!


```sql
DELIMITER $$

CREATE TRIGGER auto_insert_department
AFTER INSERT
ON employee
FOR EACH ROW
BEGIN
  DECLARE dept_name VARCHAR(255);
  IF NEW.department_id = 1 THEN
    SET dept_name = 'IT';
  ELSEIF NEW.department_id = 2 THEN
    SET dept_name = 'Sales';
  ELSEIF NEW.department_id = 3 THEN
    SET dept_name = 'HR';
  ELSE
    SET dept_name = 'Other';
  END IF;
  
  INSERT INTO department (id, Name) VALUES (NEW.department_id, dept_name);
END$$

DELIMITER ;
```

In [12]:
data.query("""
select * from employee
""")

""


In [14]:
data.execute("""
insert into employee values (101, "Jack", 90000, 1)
""")

Success!


In [9]:
data.query("""
select * from employee
""")

,e_id,Name,Salary,department_id
0,101,Jack,90000,1
1,102,Paul,80000,2


In [7]:
data.query("""
select * from department
""")

,id,Name
0,1,IT
1,2,Sales


In [8]:
data.execute("""
insert into employee values (103, "Anna", 55000, 5)
""")

Success!


In [9]:
data.commit()

Success!


In [10]:
data.query("""
select * from department
""")

,id,Name
0,1,IT
1,2,Sales
2,5,Other


**After Delete**

```sql
DELIMITER $$
CREATE TRIGGER auto_delete_department
AFTER DELETE
ON employee
FOR EACH ROW
BEGIN
  DELETE FROM department WHERE id = OLD.department_id;
END $$
DELIMITER ;
```

In [11]:
data.execute("""
DELETE FROM employee WHERE name = "anna";
""")

Success!


In [12]:
data.commit()

Success!


In [13]:
data.query("""
select * from department
""")

,id,Name
0,1,IT
1,2,Sales


In [14]:
data.execute("""
create table old_employee (
e_id int NOT NULL unique,
Name varchar(255),
department_id int)
""")

Success!


In [15]:
data.commit()

Success!


In [16]:
data.query("""
show tables;
""")

,Tables_in_triggers
0,course
1,course1
2,course_update
3,department
4,employee
5,old_employee
6,ref_course


```sql
DELIMITER $$
CREATE TRIGGER insert_into_old_employee
AFTER DELETE
ON employee
FOR EACH ROW
BEGIN
  IF NOT EXISTS (SELECT * FROM old_employee WHERE e_id = OLD.e_id) THEN
    INSERT INTO old_employee VALUES(OLD.e_id, OLD.Name, OLD.department_id);
  END IF;
END $$
DELIMITER ;
```

In [17]:
data.execute("""
insert into employee values (103, "Anna", 55000, 5)
""")

Success!


In [32]:
data.query("""
select * from department
""")

,id,Name
0,1,IT
1,2,Sales


In [31]:
data.query("""
select * from old_employee;
""")

,e_id,Name,department_id
0,103,Anna,5


In [22]:
data.commit()

Success!


In [27]:
data.execute("""
DELETE FROM employee WHERE name = "anna";
""")

Success!


In [29]:
data.commit()

Success!


In [30]:
data.query("""
select * from employee
""")

,e_id,Name,Salary,department_id
0,101,Jack,90000,1
1,102,Paul,80000,2
